In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
tf.reset_default_graph()
#embed1 = hub.Module("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1")
#embed3= hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/2")
embed3=hub.Module("https://tfhub.dev/google/elmo/2")
messages = tf.placeholder(dtype=tf.string, shape=[None])
output = embed3(messages,as_dict=True)
#tf.reset_default_graph()
session=tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])

In [ ]:
text=[]
with open("Tokens_Harry_Potter.csv") as f:
    for line in f:
        
        firstchar = line.strip()[0]
        if(firstchar in "[\""):
            line_strip =line.strip().strip("\"").strip('[]').replace("\"\"","\"")
            text.append(line_strip)
            

import nltk
sentences=nltk.sent_tokenize(" ".join(text))

In [ ]:
results=[]
for i in range(0,len(sentences)):
    results.append(session.run(output,feed_dict={messages:[sentences[i]]}))
    

In [ ]:
map_words_embeddings=[]
for sentence,result in zip(sentences,results):
    for word,embedding in zip(sentence.split(),result["elmo"][0]):
        map_words_embeddings.append({"word":word
                                    ,"embb":embedding})
        

In [ ]:
text_index = 0
with open("Tokens_Harry_Potter.csv") as f,open("Tokens_with_embedding_new.txt","w") as output:
    for line in f:
        
        firstchar = line.strip()[0]
        if(firstchar in "[\""):
            line_strip = line.strip().strip("\"").strip('[]').replace("\"\"","\"")
            text.append(line_strip)
            if(line_strip == map_words_embeddings[text_index]["word"]):
                
                print(line.strip(),",".join([str(x) for x in map_words_embeddings[text_index]["embb"]]),sep="\t",file = output)
            else:
                print("worried?")
            text_index += 1
        else:
            print(line.strip(),file = output)

In [ ]:
import pandas as pd
mrc_data=pd.read_csv("Bootstrapped_Psycholinguistic_Features.txt",sep="\t")
words_mrc={}
for i in range(1,85943):
    words_mrc[mrc_data.iloc[i].Word] = {key:mrc_data.iloc[i][key]
                                          for key in ["Familiarity"
                                                      ,"Age of Acquisition"
                                                      ,"Concreteness"
                                                      ,"Imagery"]}

In [ ]:
with open("Tokens_Harry_Potter.csv") as f,open("Tokens_with_mrc.txt","w") as output:
    for line in f:
        
        firstchar = line.strip()[0]
        if(firstchar in "[\""):
            line_strip = line.strip().strip("\"").strip('[]').replace("\"\"","\"")
            text.append(line_strip)
            word_strip=line_strip.lower().strip("\".,!?:")
            if(word_strip in words_mrc):
                
                print(line.strip()
                      ,words_mrc[word_strip]["Familiarity"]
                      ,words_mrc[word_strip]["Age of Acquisition"]
                      ,words_mrc[word_strip]["Concreteness"]
                      ,words_mrc[word_strip]["Familiarity"]
                      ,sep="\t"
                      ,file = output)
            else:
                print(line.strip()
                      ,"NA"
                      ,"NA"
                      ,"NA"
                      ,"NA"
                      ,sep="\t"
                      ,file = output)
                print("worried?",line_strip)
            text_index += 1
        else:
            print(line.strip(),file = output)